In [4]:
import requests
import pandas as pd
from pathlib import Path

ENTRY_ID = 1451223                # ← your entry ID
hist = requests.get(
    f"https://fantasy.premierleague.com/api/entry/{ENTRY_ID}/history/"
).json()

# Convert to DataFrame and save
df_summary = pd.DataFrame(hist["past"])
df_summary.to_csv("../data/season_summary.csv", index=False)
print("Season summaries saved.")


Season summaries saved.


In [5]:
ENTRY_ID    = 1234567            # ← your entry ID
SEASON_NAME = "2024/25"          # season label in history["past"]
OUT_DIR     = Path("../data") / SEASON_NAME
OUT_DIR.mkdir(parents=True, exist_ok=True)

# ─── 1) FETCH RAW DATA ─────────────────────────────────────────────────────────
# 1a) Season summary + chips
history = requests.get(
    f"https://fantasy.premierleague.com/api/entry/{ENTRY_ID}/history/"
).json()
season = next(s for s in history["past"] if s["season_name"] == SEASON_NAME)
chips  = pd.DataFrame(history["chips"])  # chip_name, event, total_gain

# 1b) Bootstrap for player metadata (positions, names)
bootstrap = requests.get(
    "https://fantasy.premierleague.com/api/bootstrap-static/"
).json()
elements = pd.DataFrame(bootstrap["elements"])[
    ["id","web_name","element_type"]
].rename(columns={"id":"element","web_name":"player","element_type":"position_id"})
# position mapping: 1=GK,2=DEF,3=MID,4=FWD
pos_map = {1:"GK",2:"DEF",3:"MID",4:"FWD"}

# 1c) GW–by–GW picks & points
gw_frames = []
for gw in range(1, 40):
    r = requests.get(
        f"https://fantasy.premierleague.com/api/entry/{ENTRY_ID}/event/{gw}/picks/"
    )
    if r.status_code != 200:
        break
    data = r.json()
    df   = pd.json_normalize(data["picks"])
    df["gw"]            = gw
    df["total_points"]  = data["entry_history"]["points"]
    gw_frames.append(df)
gw = pd.concat(gw_frames, ignore_index=True)

# ─── 2) COMPUTE METRICS ────────────────────────────────────────────────────────

# 2a) Overall points
overall_points = season["total_points"]

# 2b) Player selections: # of GWs selected + points
picks = (
    gw[["element","gw","multiplier","position","total_points","is_captain"]]
    .merge(elements, on="element", how="left")
)
player_stats = (
    picks.groupby(["element","player","position_id"])
    .agg(
        games_selected = ("gw","count"),
        points_accum   = ("total_points","sum"),
        times_captain  = ("is_captain","sum")
    )
    .reset_index()
)
player_stats["position"] = player_stats["position_id"].map(pos_map)

# 2c) Points by position breakdown
position_breakdown = (
    player_stats.groupby("position")
    .points_accum.sum()
    .rename_axis("position")
    .reset_index()
)

# 2d) Captain selections per GW
captains = (
    picks[picks.is_captain == True][["gw","player","total_points"]]
    .rename(columns={"player":"captain","total_points":"captain_points"})
)

# 2e) Chips used + return
# chips DataFrame already has chip_name, event (GW used), total_gain (return)
chips_used = chips.rename(columns={"event":"gw","total_gain":"chip_return"})

# ─── 3) SAVE OUTPUTS ──────────────────────────────────────────────────────────
season_df = pd.DataFrame([{
    "season": SEASON_NAME,
    "overall_points": overall_points,
    "overall_rank": season["rank"],
    "team_value": season["value"]
}])
season_df.to_csv(OUT_DIR / "season_summary.csv", index=False)
player_stats.to_csv(OUT_DIR / "player_metrics.csv", index=False)
position_breakdown.to_csv(OUT_DIR / "position_breakdown.csv", index=False)
captains.to_csv(OUT_DIR / "captain_history.csv", index=False)
chips_used.to_csv(OUT_DIR / "chips_usage.csv", index=False)

print(f"All metrics saved under {OUT_DIR}")

ValueError: No objects to concatenate